# Practica 2
***
Rodriguez Nuñez Diego Eduardo

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, when, trim, regexp_replace, udf
from pyspark.sql.types import IntegerType, FloatType, StringType, DateType

## Paso 1
Cargar los datos en un DataFrame de Spark

In [2]:
# Creamos la sesion de Spark
spark = SparkSession.builder.appName("Limpieza de datos").getOrCreate()

In [3]:
# Cargamos el dataset
df = spark.read.option("header", "true").csv("datos_sucios.csv", inferSchema=True)

In [4]:
df.show()

+----+------+----------+---------------+----------+-------------+
|  ID|   Nam|       Age|          Email| Join Date|       Salary|
+----+------+----------+---------------+----------+-------------+
| 1.0|  Juan|        25|  juan@mail.com|2021-06-15|         3000|
| 2.0| Maria|veintiséis| maria@mail.com|2022-07-20|         2500|
| 3.0|  NULL|        30|           NULL|06-08-2021|         4000|
| 4.0| Pedro|      NULL|     pedro@mail|2020/05/30|         5000|
| 5.0|   Ana|        22|   ana@mail.com|      NULL|         NULL|
| 5.0|   Ana|        22|   ana@mail.com|2022-07-20|         2500|
|NULL|Carlos|        40|carlos@mail.com|2019-12-01|Four thousand|
| 7.0|M4nuel|        35|manuel@mail.com|01-01-2021|         3.5K|
| 8.0|  Luis|   treinta|      luis@mail|2023-03-25|         6000|
| 9.0|   123|        29|      luis@mail|2023-03-25|          NaN|
+----+------+----------+---------------+----------+-------------+



## Paso 2
Renombrar columnas con errores tipográficos

In [5]:
# Obtenemos los nombres de las columnas
columnas = df.columns
print("Columnas originales: ",columnas)

Columnas originales:  ['ID', 'Nam', 'Age', 'Email', 'Join Date', 'Salary']


In [6]:
columnas_corregidas = {
    'Nam' : 'Name'
}

In [7]:
for col_err, col_corr in columnas_corregidas.items():
    if col_err in columnas:
        df = df.withColumnRenamed(col_err, col_corr)

In [8]:
df.show()

+----+------+----------+---------------+----------+-------------+
|  ID|  Name|       Age|          Email| Join Date|       Salary|
+----+------+----------+---------------+----------+-------------+
| 1.0|  Juan|        25|  juan@mail.com|2021-06-15|         3000|
| 2.0| Maria|veintiséis| maria@mail.com|2022-07-20|         2500|
| 3.0|  NULL|        30|           NULL|06-08-2021|         4000|
| 4.0| Pedro|      NULL|     pedro@mail|2020/05/30|         5000|
| 5.0|   Ana|        22|   ana@mail.com|      NULL|         NULL|
| 5.0|   Ana|        22|   ana@mail.com|2022-07-20|         2500|
|NULL|Carlos|        40|carlos@mail.com|2019-12-01|Four thousand|
| 7.0|M4nuel|        35|manuel@mail.com|01-01-2021|         3.5K|
| 8.0|  Luis|   treinta|      luis@mail|2023-03-25|         6000|
| 9.0|   123|        29|      luis@mail|2023-03-25|          NaN|
+----+------+----------+---------------+----------+-------------+



## Paso 3
Eliminar duplicados

In [9]:
df = df.dropDuplicates(['ID','Nombre', 'Edad', 'Email'])

In [10]:
df.show()

+----+------+----------+---------------+----------+-------------+
|  ID|Nombre|      Edad|          Email|     Fecha|      Salario|
+----+------+----------+---------------+----------+-------------+
|NULL|Carlos|        40|carlos@mail.com|2019-12-01|Four thousand|
| 1.0|  Juan|        25|  juan@mail.com|2021-06-15|         3000|
| 2.0| Maria|veintiséis| maria@mail.com|2022-07-20|         2500|
| 3.0|  NULL|        30|           NULL|06-08-2021|         4000|
| 4.0| Pedro|      NULL|     pedro@mail|2020/05/30|         5000|
| 5.0|   Ana|        22|   ana@mail.com|      NULL|         NULL|
| 7.0|M4nuel|        35|manuel@mail.com|01-01-2021|         3.5K|
| 8.0|  Luis|   treinta|      luis@mail|2023-03-25|         6000|
| 9.0|   123|        29|      luis@mail|2023-03-25|          NaN|
+----+------+----------+---------------+----------+-------------+



## Paso 4
Reemplazar valores incorrectos

### Reemplazamos los valores en edad

In [11]:
edad_diccionario = {
    'veintiséis' : 26,
    'treinta':30
}

In [12]:
for texto, valor in edad_diccionario.items():
    df = df.withColumn('Edad', when(col('Edad') == texto, valor).otherwise(col('Edad')))

### Manejar los valores nulos o incorrecto en el nombre

In [13]:
df.show()

+----+------+----+---------------+----------+-------------+
|  ID|Nombre|Edad|          Email|     Fecha|      Salario|
+----+------+----+---------------+----------+-------------+
|NULL|Carlos|  40|carlos@mail.com|2019-12-01|Four thousand|
| 1.0|  Juan|  25|  juan@mail.com|2021-06-15|         3000|
| 2.0| Maria|  26| maria@mail.com|2022-07-20|         2500|
| 3.0|  NULL|  30|           NULL|06-08-2021|         4000|
| 4.0| Pedro|NULL|     pedro@mail|2020/05/30|         5000|
| 5.0|   Ana|  22|   ana@mail.com|      NULL|         NULL|
| 7.0|M4nuel|  35|manuel@mail.com|01-01-2021|         3.5K|
| 8.0|  Luis|  30|      luis@mail|2023-03-25|         6000|
| 9.0|   123|  29|      luis@mail|2023-03-25|          NaN|
+----+------+----+---------------+----------+-------------+

